Практическое задание 6

КМБО-06-20, Бундуки Владислав, Вариант 4

Тема: "Популярные деревья в районах Нью-Йорка"

Исходные данные:
1)Набор данных - https://www.kaggle.com/new-york-city/ny-2015-street-tree-census-tree-data?select=2015-street-tree-census-tree-data.csv

 
Задачи:

1)Выделить районы, в которых состояние деревьев аномально хорошее или плохое;

2)Выделить наилучший район по состоянию деревьев;

3)Выделить виды деревьев (или более общие элементы классификации), обладающие наилучшими показателями «здоровья»

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt # библиотека для визуализации
import seaborn as sns
sns.set(style="white", color_codes=True)
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
data = pd.read_csv('2015-street-tree-census-tree-data.csv')

In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 683788 entries, 0 to 683787
Data columns (total 45 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   tree_id           683788 non-null  int64  
 1   block_id          683788 non-null  int64  
 2   created_at        683788 non-null  object 
 3   tree_dbh          683788 non-null  int64  
 4   stump_diam        683788 non-null  int64  
 5   curb_loc          683788 non-null  object 
 6   status            683788 non-null  object 
 7   health            652172 non-null  object 
 8   spc_latin         652169 non-null  object 
 9   spc_common        652169 non-null  object 
 10  steward           652173 non-null  object 
 11  guards            652172 non-null  object 
 12  sidewalk          652172 non-null  object 
 13  user_type         683788 non-null  object 
 14  problems          652124 non-null  object 
 15  root_stone        683788 non-null  object 
 16  root_grate        68

Нормируем числовые признаки и обрабатываем категориальные признаки:

In [3]:
import math
arr = ['block_id','tree_dbh','stump_diam','postcode','community board','cncldist','st_assem','st_senate','latitude','longitude','x_sp','y_sp']
for column in arr:
    data[column] = (data[column] - data[column].mean())/(math.sqrt(data[column].var()))
    
data['state'] = np.where(data['state'] == 'New York', 1, 0)

#Расположено ли дерево вдоль бордюра или со смещением от него
data['curb_loc'] = np.where(data['curb_loc'] == 'OnCurb', 1, 0)

#Повреждение тратуара в непосредственной близости от дерева
data['sidewalk'] = np.where(data['sidewalk'] == 'Damage', 1, 0)

#Указывает на наличие проблемы с корнями, вызванной брусчаткой в ложе дерева
data['root_stone'] = np.where(data['root_stone'] == 'Yes', 1, 0)

#Указывает на наличие проблемы с корнями, вызванной металлическими решетками в ложе дерева
data['root_grate'] = np.where(data['root_grate'] == 'Yes', 1, 0)

#Указывает на наличие других корневых проблем
data['root_other'] = np.where(data['root_other'] == 'Yes', 1, 0)

#Указывает на наличие проблемы со стволом, вызванной проволокой или веревкой, обмотанной вокруг ствола
data['trunk_wire'] = np.where(data['trunk_wire'] == 'Yes', 1, 0)

#Указывает на наличие проблем со стволом, вызванных установленным на дереве освещением
data['trnk_light'] = np.where(data['trnk_light'] == 'Yes', 1, 0)

#Указывает на наличие других проблем с магистралью
data['trnk_other'] = np.where(data['trnk_other'] == 'Yes', 1, 0)

#Указывает на наличие проблемы с ветвями, вызванной светильниками (обычно струнными) или проводами в ветвях
data['brch_light'] = np.where(data['brch_light'] == 'Yes', 1, 0)

#Указывает на наличие проблемы с ветками, вызванной кроссовками в ветках
data['brch_shoe'] = np.where(data['brch_shoe'] == 'Yes', 1, 0)

#Указывает на наличие других проблем с ветвями
data['brch_other'] = np.where(data['brch_other'] == 'Yes', 1, 0)

#Город, полученный из почтового индекса
my_set = set(data.zip_city)
i = 0
for item in my_set:
    data['zip_city'] = data['zip_city'].replace(item, i)
    i = i + 1

#Это название NTA, соответствующее области табуляции района по переписи населения США 2010 года, в которую попадает точка дерева
my_set = set(data.nta_name)
i = 0
for item in my_set:
    data['nta_name'] = data['nta_name'].replace(item, i)
    i = i + 1

#Это код NTA, соответствующий району, в который попадает точка дерева, согласно переписи населения США 2010 года.
my_set = set(data.nta)
i = 0
for item in my_set:
    data['nta'] = data['nta'].replace(item, i)
    i = i + 1

#Название района, в котором находится точка произрастания деревьев
my_set = set(data.borough)
i = 0
for item in my_set:
    data['borough'] = data['borough'].replace(item, i)
    i = i + 1

#Город, полученный из почтового индекса. Часто (но не всегда) это то же самое, что и район.
my_set = set(data.postcode)
i = 0
for item in my_set:
    data['postcode'] = data['postcode'].replace(item, i)
    i = i + 1

#Указывает, присутствует ли ограждение, и считает ли пользователь, что это полезное или вредное ограждение. Не регистрируется для мертвых деревьев и пней.
my_set = set(data.guards)
i = 0
for item in my_set:
    data['guards'] = data['guards'].replace(item, i)
    i = i + 1

#Это поле описывает категорию пользователя, который собрал данные этой точки дерева.
my_set = set(data.user_type)
i = 0
for item in my_set:
    data['user_type'] = data['user_type'].replace(item, i)
    i = i + 1    
    
#Указывает количество уникальных признаков бережного отношения, отмеченных для данного дерева. Не регистрируется для пней или мертвых деревьев.
my_set = set(data.steward)
i = 0
for item in my_set:
    data['steward'] = data['steward'].replace(item, i)
    i = i + 1

#Общее название вида, например, "красный клен"
my_set = set(data.spc_common)
i = 0
for item in my_set:
    data['spc_common'] = data['spc_common'].replace(item, i)
    i = i + 1
    
#Научное название вида, например, "Acer rubrum".
my_set = set(data.spc_latin)
i = 0
for item in my_set:
    data['spc_latin'] = data['spc_latin'].replace(item, i)
    i = i + 1


#Здоровье деревьев    
my_set = set(data.health)
i = 0
for item in my_set:
    data['health'] = data['health'].replace(item, i)
    i = i + 1    
    
data.loc[data.health == 'NaN', 'health'] = 0      
data.loc[data.health == 0, 'health'] = 3       
    
#Указывает, является ли дерево живым, стоящим мертвым или пнем.
data.loc[data.status == 'Alive', 'status'] = 2
data.loc[data.status == 'Stump', 'status'] = 1
data.loc[data.status == 'Dead', 'status'] = 0

data.loc[data.spc_latin == 'NaN', 'spc_latin'] = 0
data.loc[data.spc_common == 'NaN', 'spc_common'] = 0
data.loc[data.steward == 'NaN', 'steward'] = 0
data.loc[data.guards == 'NaN', 'guards'] = 0
data.loc[data.sidewalk == 'NaN', 'sidewalk'] = 0

In [4]:
#исключим столбец с уникальными значениями: tree_id
data = data.drop('tree_id', axis = 1)

#исключим столбец, включающий результаты других столбцов
data = data.drop('problems', axis = 1)

#исключим столбец с индивидуальными параметрами
data = data.drop('created_at', axis = 1)

#исключим столбец с индивидуальными параметрами
data = data.drop('address', axis = 1)

#исключим столбцы с признаками, которые отсутствуют в документации
data = data.drop('council district', axis = 1)
data = data.drop('census tract', axis = 1)
data = data.drop('bin', axis = 1)
data = data.drop('bbl', axis = 1)

ЗАДАЧА 1. Выделить районы, в которых состояние деревьев аномально хорошее или плохое.

ЗАДАЧА 2. Выделить наилучший район по состоянию деревьев.

In [8]:
#Создадим новый признак - problems {тип: число}, состоит из суммы положительных ответов о проблемах корней 
#Чем больше значение problems - тем состояние дерева хуже
problems = []
problems = (data['root_stone'] + data['root_grate'] + data['root_other'] + data['trunk_wire'] + data['trnk_light'] + data['trnk_other'] + data['brch_light'] + data['brch_shoe'] + data['brch_other'])

0         0
1         1
2         0
3         1
4         1
         ..
683783    0
683784    0
683785    0
683786    0
683787    0
Length: 683788, dtype: int32

In [9]:
problems.value_counts()

0    457944
1    162552
2     47750
3     12716
4      2524
5       249
6        49
7         2
9         1
8         1
dtype: int64

In [12]:
#Пройдем все районы и посмотрим на состояние деревьев в них
res = []
i = 0
for column in data['borough']:
    res[column] += problems.components_[i]
    i+=1

IndexError: list index out of range